# Smart Home Energy Management 
First attempt at getting a working model. 

In [1]:
using POMDPs
using POMDPToolbox
using POMDPModelTools
using Random, Distributions
using POMDPSimulators
using POMDPPolicies


#rng = MersenneTwister(1234) # TODO: Make each instance of 'rng' a different seed 
#rng = AbstractRNG; 

┌ Info: Precompiling POMDPs [a93abf59-7444-517b-a68a-c42f96afdd7d]
└ @ Base loading.jl:1242
┌ Warning: Importing POMDPToolbox. As of Julia 0.7 POMDPToolbox has been discontinued. Please use POMDPModelTools, POMDPPolicies, BeliefUpdaters, and/or POMDPSimulators instead.
└ @ POMDPToolbox /Users/coreyshono/.julia/packages/POMDPToolbox/OdZy7/src/POMDPToolbox.jl:4
┌ Info: Precompiling POMDPModelTools [08074719-1b2a-587c-a292-00f91cc44415]
└ @ Base loading.jl:1242
┌ Info: Recompiling stale cache file /Users/coreyshono/.julia/compiled/v1.2/POMDPSimulators/i1HOp.ji for POMDPSimulators [e0d0a172-29c6-5d4e-96d0-f262df5d01fd]
└ @ Base loading.jl:1240


## Global Model Parameters 

First, try to get an extremely basic version of the model running. See SIMPLE MODEL DATA PARAMS for details 

### Real Model Data
Need to write function to read in arrays of all the load/solar, occupancy, etc. from CSV files 

In [2]:
# Load Real Data Model Params 

# Import Data from CSV 
function import_data(fileapth)
    # TODO: Write import function 
    # return D, OCC, TOD, t, ODT, TOU
end


# Battery Params 
SOC_MAX = 13500; # Wh 
C_MAX = 5000; # Wh/hr 
C_RES = 100 # Wh (granularity of space) 

# Thermal Params 
TCMFT_LO = 67 # Lower thermal comfort band temp 
TCMFT_HI = 71 # Upper thermal comfort band temp 

# Building Params 
D_MAX = 5000; # Wh/hr 
D_RES = 100 # Wh (granularity of space) 


100

### SIMPLE Model Data

In [3]:
# Simple Model Params 

D_ = [3,5,3,9,8,7,5,4,3,2]; # demand (non-hvac) 
OCC = [1,0,0,1,1,1,0,0,0,1]; # occupancy 
TOD = [1,2,3,4,5,1,2,3,4,5]; # time of day 
t = collect(1:10); # time index 
ODT = [3,2,3,6,5,4,3,2,4,3]; # outdoor temp 
TOU = [2,2,3,4,2,2,2,3,4,2]; # time of use rate 


## States
Data container representing the state of the smarthome 

In [4]:
struct SmartHomeState
    d_hv::Int64 # demand of hvac (Wh, rounded to nearest 100Wh) 
    d_::Int64 # demand of all other loads (Wh, rounded to nearest 100Wh)
    soc::Int64 # state of charge of battery (Wh, rounded to nearest 100 Wh) 
    rmt::Int64 # room temperature (degF, to nearest degree) 
    occ::Bool # occupancy status 
    hsp::Int64 # heating setpoint (degF, to nearest degree) 
    csp::Int64 # cooling setpoint (degF, to nearest degree) 
    tod::Int64 # time of day (hr, 0-23) 
    odt::Int64 # outdoor temp (degF, to nearest degree) 
    tou::Int64 # time of use rate ($0.01, to nearest cent) 
    t::Int64 # current time index 
end

### TODO: Initalize State
 


Convenience functions for working with the SmartHome state 

In [5]:
# initial state constructor 
#SmartHomeState(d_hv::Int64, d_::Int64, rmt::Int64, occ::Bool, hsp::Int64, csp::Int64, tod::Int64, odt::Int64, tou::Int64) = SmartHomeState(d_hv, d_, 0.5*SOC_MAX, rmt, occ, hsp, csp, tod, odt, tou)

# checks if the setpoints of two states are the same 
#spequal(s1::SmartHomeState, s2::SmartHomeState) = s1.hsp == s2.hsp && s1.csp == s2.csp 

## Actions
Actions are defined by both the charge rate of tha battery and the change in thermostat setpoint 

In [6]:
struct SmartHomeAction
    c::Int64 # Charge rate of battery (Wh, to nearest 100 Wh) 
    dhsp::Int64 # Change in heat setpoint (degF, to nearest degree) 
    dcsp::Int64 # Change in cool setpoint (degF, to nearest degree) 
end

## MDP 
SmartHome data container is defined. It holds all the information needed to define the MDP tuple (S, A, T, R). 


In [7]:
# the smarthome mdp type 
struct SmartHomeMDP <: MDP{SmartHomeState, SmartHomeAction} 
    # Note that our MDP is parametrized by the state and the action
    # Given Data 
    D_::Array{Int64}
    OCC::Array{Bool}
    TOD::Array{Int64}
    t::Array{Int64}
    ODT::Array{Int64}
    TOU::Array{Int64} 
    
    # Battery Params
    soc_max::Int64 
    c_max::Int64
    penalty_soc::Float64
    
    # Comfort Params 
    tcomf_lo::Int64
    tcomf_hi::Int64
    prob_sp_adj::Float64
    penalty_discomf::Float64 
    
    # Building Params 
    d_max::Int64 
    penalty_sp::Float64 
    
    # Init Params 
    #init_d_hv = 5; 
    #init_soc = 5; 
    #init_rmt = 5; 
    #init_occ = true; 
    #init_hsp = 4; 
    #init_csp = 6; 
    #init_tod = 1; 
    #init_t = 1; 
    
end

SmartHomeMDP() = SmartHomeMDP(D_, OCC, TOD, t, ODT, TOU, 10, 5, -1000, 4, 6, 0.3, -1, 10, -1000); 

### TODO: Define Action Space 
Define the action space c=collect(-5:5), dhsp=collect(-5:5), dcsp=collect(-5:5)

In [8]:
#actions(::SmartHomeMDP) = SmartHomeAction(
#    collect(-5:5), # Charge rate of battery 
#    collect(-5:5), # Change in heat setpoint
#    collect(-5:5)  # Change in cool setpoint
#)

## Define gen 
Implement the complete generative models for both the SmartHomeMDP and SmartHomePOMDP. 

### State transition model 
- d_hv: f(hsp, csp, odt) 
- d_: probabilistic f(d_(t-1), tod, rand)
- soc: f(soc(t-1), c(t-1)) 
- rmt: f(hsp, csp, odt) 
- occ: probabilistic f(tod, rand) 
- hsp: f(hsp(t-1), dhsp, rand) 
- csp: f(csp(t-1), dcsp, rand) 
- tod: f(tod(t-1))
- odt: f(odt(t-1), tod) 
- tou: f(tod) 

### Observation model 
- occ_d: f(occ, p_tp, p_fp) 
    - p_tp: true pos prob, 
    - p_fp: false pos prob
    
### Reward model
Reward model is composed of the following components: 
- TOU charge demand_: f(tou, d_) 
- TOU charge on demand_hvac: f(tou, d_hv) 
- TOU charge on charge_rate: f(tou, c) 
- Discomfort Penalty: f(occ, rmt) 
- SOC violation Penalty: f(soc)  
- SP violation penalty: f(hsp, csp) 
- Demand Charge: f(D_, D_HV, C) depends on all timesteps for the entire duration 


In [56]:
using Distributions

In [59]:
rand(Uniform(1,10))

UndefVarError: UndefVarError: Uniform not defined

In [60]:
function update_sps(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    if a.dhsp != 0 || a.dcsp != 0 
        # If action=adjust, deterministically set thermostat 
        return s.hsp + a.dhsp, s.csp + a.dcsp
    else
        # Else, probabilistic adjustment based on thermal comfort 
        if rand(rng) < m.prob_sp_adj
            # Adjust setpoints 
            #septoints = [round(x) for x in rand(rng, Uniform(m.tcomf_lo, m.tcomf_hi),2)] # TODO: Figure out why not working 
            center = (m.tcomf_lo + m.tcomf_hi)/2
            setpoints = [round(x) for x in min(max(rand(rng, Normal(center,2)),m.tcomf_lo),m.tcomf_hi)]
            return min(setpoints), max(setpoints) 
        else
            # Maintain previous SPs
            return s.hsp, s.csp 
        end
    end
end

function hv_model(hsp, csp, odt, rng) 
    # Fitted regression model based hv_model data analysis. 
    # Model was fit to predict kWh/day 
    # d_hv = a*cdd + b*hdd + c + N(0,RMSE)  
    
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 2.1898; b = 0.9476; c = 0.5394; RMSE = 6.146; 
    error = rand(rng, Normal(0,RMSE), 1) 
    
    d_hv = a*cdd + b*hdd + c + error
    d_hv = round(d_hv*1000/24, digits=-2) # kWh/day -> Wh/hr(to nearest 100Wh) 
    
    return d_hv
end

function hv_model_simple(hsp, csp, odt, rng)
    # Extremely simplified model to work with SIMPLE DATA 
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 2.3; b = 1.7; RMSE = 0.8; 
    error = rand(rng, Normal(0,RMSE), 1)[1]
    
    d_hv = a*cdd + b*hdd + error 
    d_hv = max(min(round(d_hv),10), 0) 
    
    return d_hv
end

function update_d_(d_, tod)
    # TODO: fit model dependent on TOD
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    noise_d_ = 3
    
    d_ += rand(rng, Normal(0,noise_d_), 1)[1] # Probabilistic Change 
    d_ = max(min(round(d_),10), 0) # Clamp to 0-10 int
    return d_
end

function update_occ(occ, tod)
    # TODO: fit model dependent on TOD 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    p_change_occ = 0.3
    
    return rand(rng) < p_change_occ ? !occ : occ
end

function update_odt(tod)
    # Fitted to a sine wave with noise 
    # TODO: need to adjust equation params when converting to realistic data 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # TODO: fit model dependent on TOD 
    # Maybe fit a BN to this data? 
    noise_odt = 1
    
    odt = 2.5*sin((tod+1.75)*pi/2.5)+5
    odt += rand(rng, Normal(0,noise_odt), 1)[1] 
    odt = max(min(round(odt),10), 0) # Clamp to 0-10 int 
    
    return odt
end

function update_tou(tod) 
    TOU_SCHEDULE = [2,2,3,4,2]; 
    # TODO: Change tou_schedule to take as model input rather than hard code 
    tou = TOU_SCHEDULE[tod] 
    return tou 
end


# MDP Generative Model 
function POMDPs.gen(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    t = s.t + 1 # Deterministic, fixed 
    tod = rem(s.tod + 1, 24) # Deterministic, fixed 
    tod = rem(s.tod + 1, 5) + 1 # TODO: THIS IS FOR SIMPLE MODEL 
    
    #d_ = m.D_[s.t+1] # Deterministic, fixed (should/could it be probabilistic?)
    #occ = OCC[s.t+1] # Deterministic, fixed (should/could it be probabilistic?) 
    #odt = ODT[s.t+1] # Deterministic, fixed 
    #tou = TOU[s.t+1] # Deterministic, fixed 
    
    d_ = update_d_(s.d_, tod) # Probabilistic
    occ = update_occ(s.occ, tod) # Probabilistic
    odt = update_odt(tod) # Probabilistic
    tou = update_tou(tod) # By Schedule
    
    
    (hsp, csp) = update_sps(m, s, a, rng) 
    #d_hv = hv_model(hsp, csp, odt, rng) # Based on actual fit 
    d_hv = hv_model_simple(hsp, csp, s.odt, rng) 
    soc = s.soc + a.c
    rmt = min(max(hsp,odt),csp)
    
    sp = SmartHomeState(d_hv, d_, soc, rmt, occ, hsp, csp, tod, odt, tou, t)
    

    # observation model 
    # N/A
    
    # reward model 
    r = tou * (s.d_ + s.d_hv + a.c)
    r += (s.rmt > m.tcomf_hi || s.rmt < m.tcomf_lo) ? m.penalty_discomf : 0 
    r += (s.soc > m.soc_max || s.soc < 0) ? m.penalty_soc : 0 
    r += (s.hsp > s.csp) ? m.penalty_sp : 0 # HSP must be less than or equal to CSP 
    
    
    # create and return a NamedTuple 
    return (sp=sp, r=r) # For MDP 
end



### TODO: Convert above code from MDP -> POMDP

In [10]:
# POMDP Generative Model 
function POMDPs.gen(m::SmartHomePOMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    # should be same as MDP 
    
    # observation model 
    if sp # Opccupied 
        o = rand(rng) < m.p_occ_tp
    else # Not Occupied 
        o = rand(rng) < m.p_occ_fp
    end
    
    
    # reward model 
    
    
    # create and return a NamedTuple 
    return (sp=sp, o=o, r=r) # For POMDP 
end



UndefVarError: UndefVarError: SmartHomePOMDP not defined

## Initial State Distribution
At first, lets assume a somewhat arbitrary assignment of deterministic states and assume the transition model will handle the rest. 


In [11]:
#POMDPs.initialstate_distribution(m::BabyPOMDP) = Deterministic(false)
#POMDPs.initialstate_distribution(m::SmartHomeMDP) = SmartHomeState(
#    Deterministic(5), # d_hv
#    Deterministic(5), # d_
#    Deterministic(5), # soc 
#    Deterministic(5), # rmt
#    Deterministic(true), # occ
#    Deterministic(4), # hsp
#    Deterministic(6), # csp
#    Deterministic(1), # tod
#    Deterministic(5), # odt
#    Deterministic(2), # tou
#    Deterministic(1) # t 
#) 

#POMDPs.initialstate_distribution(m::SmartHomeMDP) = SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)  

In [12]:
sh = SmartHomeMDP(); 

## Step Through Random Policy 

In [32]:
POMDPs.initialstate(m::SmartHomeMDP, rng::AbstractRNG) = POMDPs.initialstate(::Union{MDP, POMDP}, ::AbstractRNG)

initialstate(m::SmartHomeMDP, rng::AbstractRNG) = SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)  


function POMDPs.initialstate_distribution(pp::UAVchaseProblem)
    p = mdp(pp)
    init_uav_pose = p.init_UAVPose
    init_uav_heading = p.init_UAVHeading
    init_taget_pose = p.init_targetPose
    init_target_std = p.init_target_std
    return MDPStateDistribution(init_uav_pose, init_uav_heading, init_taget_pose, init_target_std)
end

function POMDPs.initialstate_distribution(m::SmartHomeMDP) 
    p = mdp(pp)
    init_uav_pose = p.init_UAVPose
    init_uav_heading = p.init_UAVHeading
    init_taget_pose = p.init_targetPose
    init_target_std = p.init_target_std
    return MDPStateDistribution(init_uav_pose, init_uav_heading, init_taget_pose, init_target_std)
end

ErrorException: syntax: invalid "::" syntax

In [ ]:
n_actions(p::TruckMaintenance) = 2
n_states(p::TruckMaintenance)  = 2

ordered_states(p::TruckMaintenance)                        = states(p)
initialstate_distribution(p::TruckMaintenance)             = SparseCat(states(p), [1.0, 0.0])

states(p::TruckMaintenance)                                = [TruckState(false, 0.0),     TruckState(true, 0.0)]
states(p::TruckMaintenance, s::TruckState)                 = [TruckState(false, s.d),     TruckState(true, s.d)]
states(p::TruckMaintenance, s::TruckState, a::TruckAction) = [TruckState(false, s.d+a.d), TruckState(true, s.d+a.d)]

actions(p::TruckMaintenance)                               = [TruckAction(false, rand(50:50:1500)), TruckAction(true, 0)]

stateindex(p::TruckMaintenance, s::TruckState)   = 2 - s.fault
actionindex(p::TruckMaintenance, a::TruckAction) = 2 - a.repair

In [26]:
#n_actions(m::Smart) = 
POMDPs.initialstate(m::SmartHomeMDP, rng::MersenneTwister) = SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)  
POMDPs.initialstate_distribution(m::SmartHomeMDP) = SparseCat([SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1), SmartHomeState(4, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)], [0.4, 0.6])

POMDPs.actions(m::SmartHomeMDP) = [SmartHomeAction(-1,0,0), SmartHomeAction(0,0,0), SmartHomeAction(1,0,0), 
    SmartHomeAction(-1,1,0), SmartHomeAction(0,1,0), SmartHomeAction(1,1,0), 
    SmartHomeAction(-1,0,1), SmartHomeAction(0,0,1), SmartHomeAction(1,1,1)]  

#actions(::SmartHomeMDP) = SmartHomeAction(
#    collect(-5:5), # Charge rate of battery 
#    collect(-5:5), # Change in heat setpoint
#    collect(-5:5)  # Change in cool setpoint
#)



In [20]:
rng = Random.GLOBAL_RNG; 

In [61]:
policy = RandomPolicy(sh)

for (s, a, r) in stepthrough(sh, policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
    println()
    #println("TOD: $s.tod")
end

s = SmartHomeState(5, 5, 5, 5, true, 4, 6, 1, 5, 2, 1)
a = SmartHomeAction(0, 0, 1)
r = 30

s = SmartHomeState(0, 6, 5, 5, true, 4, 7, 3, 5, 3, 2)
a = SmartHomeAction(1, 1, 1)
r = 14

s = SmartHomeState(0, 6, 6, 5, true, 5, 8, 5, 5, 2, 3)
a = SmartHomeAction(0, 0, 1)
r = 12

s = SmartHomeState(0, 7, 6, 5, false, 5, 9, 2, 1, 2, 4)
a = SmartHomeAction(1, 1, 0)
r = 32

s = SmartHomeState(8, 10, 7, 8, false, 6, 9, 4, 8, 4, 5)
a = SmartHomeAction(1, 1, 0)
r = 37.0

s = SmartHomeState(1, 8, 8, 7, false, 7, 9, 1, 4, 2, 6)
a = SmartHomeAction(1, 1, 1)
r = 29.0

s = SmartHomeState(6, 6, 9, 8, false, 8, 10, 3, 4, 3, 7)
a = SmartHomeAction(-1, 1, 0)
r = 21.0

s = SmartHomeState(8, 7, 8, 9, false, 9, 10, 5, 7, 2, 8)
a = SmartHomeAction(1, 1, 0)
r = 31.0

s = SmartHomeState(5, 7, 9, 10, true, 10, 10, 2, 2, 2, 9)
a = SmartHomeAction(-1, 1, 0)
r = 43.0

s = SmartHomeState(10, 5, 8, 10, true, 11, 10, 4, 7, 4, 10)
a = SmartHomeAction(1, 1, 0)
r = -969.0



## Solve MDP 

In [62]:
using BasicPOMCP # For the solver

┌ Info: Precompiling BasicPOMCP [d721219e-3fc6-5570-a8ef-e5402f47c49e]
└ @ Base loading.jl:1242


In [63]:
# Define the POMCP solver; use keyword arguments to adjust parameters
solver = POMCPSolver(c=10.0)

POMCPSolver
  max_depth: Int64 20
  c: Float64 10.0
  tree_queries: Int64 1000
  max_time: Float64 Inf
  tree_in_info: Bool false
  default_action: ExceptionRethrow ExceptionRethrow()
  rng: MersenneTwister
  estimate_value: RolloutEstimator
